In [44]:
import pandas as pd
import polars as pl
from prophet import Prophet
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import cross_validation, performance_metrics
from arxiv_categories import physics_categories, math_categories, cs_categories, astro_categories


from tqdm import tqdm
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import os


In [ ]:

def format_pubs_by_week(filepath, subdiscipline_code):

    # Load the CSV file with schema
    schema = {
        "id": pl.Utf8,
        "authors": pl.Utf8,
        "title": pl.Utf8,
        "categories": pl.Utf8,
        "comments": pl.Utf8,
        "update_date": pl.Date,  # Parsing standard date format
        "latest_created_date": pl.Utf8,  # Read as string first for manual parsing
    }
    
    # Read the CSV file with schema
    try:
        raw_df = pl.read_csv(filepath, schema=schema)
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

    # Parse 'latest_created_date' to Datetime with specific format
    df = raw_df.with_columns(
        pl.col("latest_created_date").str.strptime(
            pl.Datetime("us"), 
            format="%a, %d %b %Y %H:%M:%S %Z",  # Using 'format' instead of 'fmt'
            strict=False  # Allows partial parsing or missing values
        )
    ).drop_nulls(subset=["latest_created_date"])

    # Truncate to weeks
    df = df.with_columns(
        (pl.col("latest_created_date").dt.truncate("1w")).alias("year_week")
    )

    # Filter the DataFrame for the specified subdiscipline code - using regex for exact match
    pattern = fr"\b{subdiscipline_code}\b"
    df = df.filter(pl.col("categories").str.contains(pattern, literal=False))

    # print("Checking unique weeks detected:")
    # print(df.select(pl.col("year_week").unique()))

    # Count publications per week
    weekly_counts = df.group_by("year_week").agg([
        pl.col("id").count().alias("publications_per_week")
    ])

    # Adding the keyword to the DataFrame
    weekly_counts = weekly_counts.with_columns(
        pl.lit(subdiscipline_code).alias("keyword")
    )

    weekly_counts = weekly_counts.sort("year_week")

    # Return the final DataFrame
    return weekly_counts



df = format_pubs_by_week(filepath="cs_arxiv_data.csv", subdiscipline_code="cs.CG")
field_prophet_df = df.to_pandas()
field_prophet_df = field_prophet_df[["year_week", "publications_per_week"]]
field_prophet_df = field_prophet_df.rename(columns={'year_week': 'ds', 'publications_per_week': 'y'})
#print(field_prophet_df)

In [ ]:

def evaluate_prophet(field_prophet_df, changepoint_val=0.01, seasonality_val=0.1):

    #Fit
    model = Prophet(
        changepoint_prior_scale=changepoint_val,
        seasonality_prior_scale=seasonality_val
    )

    model.fit(field_prophet_df)

    df_cv = cross_validation(model, initial='3650 days', horizon='365 days', period='180 days')

    df_performance = performance_metrics(df_cv)

    # Retrieve the RMSE metric and add it to the results DataFrame
    rmse = df_performance['rmse'].mean()

    results = results.append({
            'changepoint_prior_scale': changepoint_val,
            'seasonality_prior_scale': seasonality_val,
            'rmse': rmse
        }, ignore_index=True)


    # future_field = field.make_future_dataframe(periods=52)
    # future_field.tail()

    # forecast_field = field.predict(future_field)
    # forecast_field[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

    # fig1 = field.plot(forecast_field)
    # fig2 = field.plot_components(forecast_field)

    # df_cv = cross_validation(
    # model=field,
    # initial='3650 days',  # Initial training period (e.g., 10 years of weekly data)
    # horizon='365 days',   # Forecasting horizon (e.g., 1 year)
    # period='180 days'     # Spacing between cutoff dates (e.g., every 6 months)
    # )

    # df_performance = performance_metrics(df_cv)
    # print(df_performance.head())

    
    fig1 = plot_cross_validation_metric(df_cv, metric='coverage')
    fig2 = plot_cross_validation_metric(df_cv, metric='rmse')


    #Evaluate

evaluate_prophet(field_prophet_df)

03:22:21 - cmdstanpy - INFO - Chain [1] start processing
03:22:21 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/15 [00:00<?, ?it/s]

03:22:21 - cmdstanpy - INFO - Chain [1] start processing
03:22:21 - cmdstanpy - INFO - Chain [1] done processing
03:22:21 - cmdstanpy - INFO - Chain [1] start processing
03:22:22 - cmdstanpy - INFO - Chain [1] done processing
03:22:22 - cmdstanpy - INFO - Chain [1] start processing
03:22:22 - cmdstanpy - INFO - Chain [1] done processing
03:22:22 - cmdstanpy - INFO - Chain [1] start processing
03:22:22 - cmdstanpy - INFO - Chain [1] done processing
03:22:22 - cmdstanpy - INFO - Chain [1] start processing
03:22:22 - cmdstanpy - INFO - Chain [1] done processing
03:22:22 - cmdstanpy - INFO - Chain [1] start processing
03:22:22 - cmdstanpy - INFO - Chain [1] done processing
03:22:23 - cmdstanpy - INFO - Chain [1] start processing
03:22:23 - cmdstanpy - INFO - Chain [1] done processing
03:22:23 - cmdstanpy - INFO - Chain [1] start processing
03:22:23 - cmdstanpy - INFO - Chain [1] done processing
03:22:23 - cmdstanpy - INFO - Chain [1] start processing
03:22:23 - cmdstanpy - INFO - Chain [1]

UnboundLocalError: cannot access local variable 'results' where it is not associated with a value

In [50]:
def hyperparameter_tuning(field_prophet_df):
    changepoint_prior_scales = [0.01, 0.1, 0.5]
    seasonality_prior_scales = [1.0, 10.0]

    results = []  # Store results in a list for better performance

    for changepoint_val in changepoint_prior_scales:
        for seasonality_val in seasonality_prior_scales:
            print(f"Training with changepoint_prior_scale={changepoint_val}, seasonality_prior_scale={seasonality_val}")
            
            # Initialize and fit the model
            model = Prophet(
                changepoint_prior_scale=changepoint_val,
                seasonality_prior_scale=seasonality_val
            )
            
            model.fit(field_prophet_df)

            # Cross-validation
            df_cv = cross_validation(
                model,
                initial='4380 days',  # 12 years of data
                horizon='365 days',   # 1 year forecast window
                period='180 days'     # Training window rolls forward every 6 months
            )

            # Performance metrics
            df_performance = performance_metrics(df_cv)
            rmse = df_performance['rmse'].mean()  # Calculate average RMSE

            # Store the results
            results.append({
                'changepoint_prior_scale': changepoint_val,
                'seasonality_prior_scale': seasonality_val,
                'rmse': rmse
            })

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Find the best parameters
    best_params = results_df.loc[results_df['rmse'].idxmin()]
    print("\nBest Parameters:")
    print(best_params)

    return results_df

# Assuming your field_prophet_df is already prepared
#results_df = hyperparameter_tuning(field_prophet_df)


           

In [52]:
def tune_all_hyperparameters(discipline, categories):
    best_params_list = []

 
    with tqdm(total=len(categories), desc=f"Tuning {discipline} subdisciplines", ncols=100) as pbar:
        for category_name in categories:

            try:
                # Update the progress bar description
                pbar.set_postfix_str(f"Processing {category_name}")

                # Load the category dataframe
                category_df = format_pubs_by_week(filepath=f"{discipline}_arxiv_data.csv", subdiscipline_code=category_name)
                category_df = category_df.to_pandas()
                category_df = category_df[["year_week", "publications_per_week"]]
                category_df = category_df.rename(columns={'year_week': 'ds', 'publications_per_week': 'y'})

                # Tune hyperparameters
                best_params = hyperparameter_tuning(category_df)
                best_params['category_name'] = category_name  # Keep track of which category this result is from
                best_params_list.append(best_params)
                
            except Exception as e:
                print(f"\nFailed to process category {category_name}: {e}")
            
            # Update progress bar after processing each category
            pbar.update(1)
    
    # Convert the results list to a DataFrame
    best_params_df = pd.DataFrame(best_params_list)

    # Calculate the mean of the best hyperparameters across all subdisciplines
    mean_changepoint_prior_scale = best_params_df['changepoint_prior_scale'].mean()
    mean_seasonality_prior_scale = best_params_df['seasonality_prior_scale'].mean()

    print(f"\nAverage Best changepoint_prior_scale across all categories: {mean_changepoint_prior_scale}")
    print(f"Average Best seasonality_prior_scale across all categories: {mean_seasonality_prior_scale}")
    
    return best_params_df

hyperparameters_df = tune_all_hyperparameters("cs", cs_categories)

print(hyperparameters_df)


Tuning cs subdisciplines:   0%|                            | 0/40 [00:00<?, ?it/s, Processing cs.AI]15:38:30 - cmdstanpy - INFO - Chain [1] start processing
15:38:31 - cmdstanpy - INFO - Chain [1] done processing


Training with changepoint_prior_scale=0.01, seasonality_prior_scale=1.0


  0%|          | 0/11 [00:00<?, ?it/s]

15:38:31 - cmdstanpy - INFO - Chain [1] start processing
15:38:31 - cmdstanpy - INFO - Chain [1] done processing
15:38:31 - cmdstanpy - INFO - Chain [1] start processing
15:38:31 - cmdstanpy - INFO - Chain [1] done processing
15:38:31 - cmdstanpy - INFO - Chain [1] start processing
15:38:31 - cmdstanpy - INFO - Chain [1] done processing
15:38:31 - cmdstanpy - INFO - Chain [1] start processing
15:38:31 - cmdstanpy - INFO - Chain [1] done processing
15:38:31 - cmdstanpy - INFO - Chain [1] start processing
15:38:31 - cmdstanpy - INFO - Chain [1] done processing
15:38:31 - cmdstanpy - INFO - Chain [1] start processing
15:38:31 - cmdstanpy - INFO - Chain [1] done processing
15:38:32 - cmdstanpy - INFO - Chain [1] start processing
15:38:32 - cmdstanpy - INFO - Chain [1] done processing
15:38:32 - cmdstanpy - INFO - Chain [1] start processing
15:38:32 - cmdstanpy - INFO - Chain [1] done processing
15:38:32 - cmdstanpy - INFO - Chain [1] start processing
15:38:32 - cmdstanpy - INFO - Chain [1]

Training with changepoint_prior_scale=0.01, seasonality_prior_scale=10.0


...

15:48:18 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/10 [00:00<?, ?it/s]

15:48:18 - cmdstanpy - INFO - Chain [1] start processing
15:48:19 - cmdstanpy - INFO - Chain [1] done processing
15:48:19 - cmdstanpy - INFO - Chain [1] start processing
15:48:19 - cmdstanpy - INFO - Chain [1] done processing
15:48:19 - cmdstanpy - INFO - Chain [1] start processing
15:48:19 - cmdstanpy - INFO - Chain [1] done processing
15:48:19 - cmdstanpy - INFO - Chain [1] start processing
15:48:19 - cmdstanpy - INFO - Chain [1] done processing
15:48:19 - cmdstanpy - INFO - Chain [1] start processing
15:48:19 - cmdstanpy - INFO - Chain [1] done processing
15:48:19 - cmdstanpy - INFO - Chain [1] start processing
15:48:20 - cmdstanpy - INFO - Chain [1] done processing
15:48:20 - cmdstanpy - INFO - Chain [1] start processing
15:48:20 - cmdstanpy - INFO - Chain [1] done processing
15:48:20 - cmdstanpy - INFO - Chain [1] start processing
15:48:20 - cmdstanpy - INFO - Chain [1] done processing
15:48:20 - cmdstanpy - INFO - Chain [1] start processing
15:48:21 - cmdstanpy - INFO - Chain [1]


Best Parameters:
changepoint_prior_scale     0.500000
seasonality_prior_scale     1.000000
rmse                       33.488519
Name: 4, dtype: float64


ValueError: Must pass 2-d input. shape=(40, 6, 4)